# Download a ViSAPy data file

This is the retina data set generated with (ViSAPy)[https://github.com/espenhgn/ViSAPy] as presented in Espen's (paper)[http://dx.doi.org/10.1016/j.jneumeth.2015.01.029] in the Journal of Neuroscience Methods.

In [ ]:
import urllib.request
file_url = 'https://datasync.ed.ac.uk/public.php/webdav'

password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_mgr.add_password(None, file_url, "GbnXJ5bU3m9JPau", "HS2")
handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
opener = urllib.request.build_opener(handler)
opener.open(file_url)
urllib.request.install_opener(opener)
with urllib.request.urlopen(file_url) as response, open('visapy_data.npy', 'wb') as out_file:
    data = response.read()
    out_file.write(data)

# Use of the `HSDetection` class

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from herdingspikes import HSDetection
from herdingspikes.probe import HierlmannVisapyEmulationProbe

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# raw data location
data_path = 'visapy_data.npy'

# detection parameters
to_localize = True
cutout_start = 14
cutout_end = 36
threshold = 12
file_name = 'ProcessedSpikes_visapy'

In [ ]:
Probe = HierlmannVisapyEmulationProbe(data_path)
H = HSDetection(Probe, to_localize, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name, save_all=True)

In [ ]:
Probe.show()

In [ ]:
H.DetectFromRaw()

In [ ]:
H.LoadDetected()

In [ ]:
plt.figure(figsize=(10, 10))
H.PlotTracesChannels(200, window_size=100)

In [ ]:
plt.figure(figsize=(14, 3))
ax = plt.subplot(111)
H.PlotAll(invert=True, s=1, alpha=0.2, ax=ax)
ax.set_xlim((0,500))

# Use of the `HSClustering` class

In [ ]:
from herdingspikes import HSClustering

In [ ]:
# read
C = HSClustering(H)

In [ ]:
%%time
C.ShapePCA(pca_ncomponents=2, pca_whiten=True)

In [ ]:
%%time
# note parameters are not particularly optimised (yet)
C.CombinedClustering(alpha=4, bandwidth=5., bin_seeding=False, n_jobs=-1)

In [ ]:
plt.figure(figsize=(14, 4))
ax = plt.subplot(111)
C.PlotAll(invert=True, s=1, ax=ax, max_show=100000,show_labels=False)
plt.title("MeanShift, no bin seeding");

In [ ]:
units = range(16)
C.PlotShapes(units)

In [ ]:
C.PlotNeighbourhood(1,radius=6, alpha=0.8);

In [ ]:
C.SaveHDF5(file_name+"_sorted.hdf5")